# Step 3: Create array of unique legislators

In [1]:
import re
import os
import numpy as np
import pandas as pd
import ujson as json
from tqdm import tqdm
import sqlCommands

from dateutil.parser import parse as DateParser



def findOccurences(s, ch):
    res = list()
    for i in re.finditer(ch,s):
        res.append(i.start())
    return res

In [2]:
#set up engine
dbname = 'legislatr'
engine = sqlCommands.get_engine(dbname)
#query = "SELECT * FROM topics;"
bills_df = pd.read_sql_table('allbills4',engine)

sponsor = list(bills_df['sponsor'])
cosponsors = list(bills_df['cosponsors'])+sponsor
print(type(sponsor))
print(type(sponsor[0]))
print(sponsor[0:10])
print(cosponsors[0:10])

<class 'list'>
<class 'str'>
['{Sen,"Baucus, Max",MT}', '{Sen,"Brown, Sherrod",OH}', '{Sen,"Kirk, Mark Steven",IL}', '{Sen,"Wyden, Ron",OR}', '{Sen,"Hirono, Mazie K.",HI}', '{Sen,"Udall, Mark",CO}', '{Sen,"Toomey, Pat",PA}', '{Sen,"Lee, Mike",UT}', '{Sen,"Schumer, Charles E.",NY}', '{Sen,"Burr, Richard",NC}']
['{}', '{{Sen,"Casey, Robert P., Jr.",PA},{Sen,"Franken, Al",MN},{Sen,"Klobuchar, Amy",MN},{Sen,"Portman, Rob",OH},{Sen,"Toomey, Pat",PA}}', '{{Sen,"Hatch, Orrin G.",UT},{Sen,"Johnson, Tim",SD},{Sen,"Murkowski, Lisa",AK}}', '{}', '{{Sen,"Gillibrand, Kirsten E.",NY},{Sen,"Markey, Edward J.",MA},{Sen,"Reed, Jack",RI},{Sen,"Sanders, Bernard",VT},{Sen,"Schumer, Charles E.",NY},{Sen,"Whitehouse, Sheldon",RI}}', '{{Sen,"Bennet, Michael F.",CO},{Sen,"Heller, Dean",NV}}', '{{Sen,"Manchin, Joe, III",WV}}', '{}', '{{Sen,"Baldwin, Tammy",WI},{Sen,"Blumenthal, Richard",CT},{Sen,"Blunt, Roy",MO},{Sen,"Cochran, Thad",MS},{Sen,"Collins, Susan M.",ME},{Sen,"Durbin, Richard",IL},{Sen,"Franken, Al"

In [3]:
#separate cosponsors and remove entries where there are no cosponsors
new_cosponsors = list()
for s in cosponsors:
    #first check for empty
    if s == "{}":
        continue
    s= s.replace("{","")
    ind = findOccurences(s,"},")
    #loop over each new sponsor
    sind = 0
    ind.append(-1)
    for i in ind:
        sub_spons = s[sind:i]
        sind = i+2
        sub_spons = sub_spons.replace("}","")
        new_cosponsors.append(sub_spons)

In [5]:
u_new_cosponsors = np.unique(new_cosponsors)
print(len(u_new_cosponsors)) #should be 865 based on playing before
print(u_new_cosponsors[-10:-1]) #too may print lines

869
['Sen,"Udall, Tom",NM' 'Sen,"Vitter, David",LA'
 'Sen,"Voinovich, George V.",OH' 'Sen,"Walsh, John E.",MT'
 'Sen,"Warner, Mark R.",VA' 'Sen,"Warren, Elizabeth",MA'
 'Sen,"Webb, Jim",VA' 'Sen,"Whitehouse, Sheldon",RI'
 'Sen,"Wicker, Roger F.",MS']


In [8]:
#use this box to check things I am suspicious about.
for s in u_new_cosponsors:
    ind = findOccurences(s,"Warner")
    if len(ind) == 0:
        continue
    print(s)

Sen,"Warner, Mark R.",VA


In [19]:
#save unique sponsor list to database
sponsor_list_df = pd.DataFrame()
sponsor_list_df['sponsor'] = u_new_cosponsors
sqlCommands.write_to_database(dbname,'sponsor_list',sponsor_list_df,engine)

# STOP HERE!

In [17]:
#try making a query for bills with a certain person in them.
test_q = u_new_cosponsors[-1]
print(test_q)
query = "SELECT bill_type, bill_number, congress FROM allbills4 WHERE sponsor LIKE '%"+test_q+"%' OR cosponsor LIKE '%"+test_q+"%' ;"
test_bill = pd.read_sql_query(query,engine)
test_bill


Sen,"Wyden, Ron",OR


TypeError: 'dict' object does not support indexing

In [11]:
foo = pd.DataFrame()

In [15]:
pd.read_sql_query?